# NJ Incarcerated Census Exploration with Python

### Step 1: Installing the Necessary Libraries

In [ ]:
%pip install mysql-connector-python
%pip install matplotlib
%pip install pandas

In [1]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as mp
import csv

### Step 2: Connecting to the SQL Database and Querying the Data
**2.1:** In this section, I originally imported the dataset into my own SQL database and queried the data from this notebook. I have kept but commented out the original code for this process and instead directly imported the csv file using read_csv. This will allow you to easily test the python code yourself without needing to create your own SQL database.

In [56]:
#cnx = mysql.connector.connect(user='root', 
#                                password='******', 
#                                host='******', 
#                                database='database1',)

#cursor = cnx.cursor()

In [57]:
#query  = ("SELECT * FROM database1.nj_ip_census")
#cursor.execute(query)

In [58]:
#response = cursor.fetchall()

In [2]:
response = pd.read_csv('NJDOC_Incarcerated_Persons_Census_20240513.csv')

### Step 3: Importing the Data into the Notebook
**3.1:** The csv data is fed into a pandas dataframe titled "df1". Column titles were shortened and spaces were replaced with underscores.

In [ ]:
df1 = pd.DataFrame(response)
df1.columns = ["ID_Number","Term_at_Admission","Total_Max_Term","Gender","County_of_Admission","Ethnicity","Age"]

print("In total, there were ", df1.shape[0], "Incarcerated persons (IPs) in New Jersey state prisons during 2023.")

df1.head(10)

### Step 4: Analyzing the Age Distribution
**4.1:** Incarcerated persons were categorized into the histogram below based on their age. The resulting graph demonstrates a right skewed distribution in which younger individuals between the ages 30 and 40 are the majority age group in the census. With the 20 to 30 group containing the second highest number of individuals.

In [ ]:
bins = [10,20,30,40,50,60,70,80,90]

mp.style.use('ggplot')
mp.hist(df1["Age"], bins=bins, edgecolor='black', color='blue')

mp.xlabel('Age')
mp.ylabel('Frequency')
mp.title('Age of Incarcerated Persons')

**4.2:** Calculating the median age.

In [ ]:
print("The median age in this dataset is", df1["Age"].median())

**4.3:** To further explore the age distribution, the following code uses a custom function to create a new variable named "Elderly_Status". If an individual is aged at or above 65 the attribute "Elderly" is added to their record. A new dataframe was created for this process to avoid makeing unwanted changes to the master data source.

In [26]:
df_age = pd.DataFrame(df1[["ID_Number", "Age"]])

def add_column(row):
    if row["Age"] >= 65: cat = "Elderly"
    else: cat = "Not_Elderly"
    return cat

df_age["Elderly_Status"] = df_age.apply(add_column, axis = 1)

**4.4:** By calculating the percentage of individuals in both categories, it becomes clear that only 3.5 percent are elderly.

In [ ]:
print(sum(df_age['Elderly_Status'] == 'Elderly') / df_age.shape[0] * 100)
print(sum(df_age['Elderly_Status'] == 'Not_Elderly') / df_age.shape[0] * 100)

**4.5:** The percentages were fed into a pie chart to visualize the difference between the number of elderly and non-elderly individuals. Percentages, colors, and data labels were made into their own lists and used as the parameters in the "mp.pie" function.

In [ ]:

labels = ['Elderly\n3.5%', 'Not Elderly\n96.5%']
colors = ['#3e4dfa', '#7ffa3e']
sizes = [3.5, 96.5]

mp.pie(sizes, labels=labels, colors=colors, shadow=True)

mp.title('Elderly Status of Incarcerated Persons')
mp.legend(title='Legend', loc='center right', bbox_to_anchor=(1, 0, 0.5, 1))

### Step 5: Exploring and Recoding Ethnicity Data

**5.1:** Finding the total number of incarcerated individuals for each ethnic group.

In [ ]:
df1["Ethnicity"].value_counts()

**5.2:** Calculating the total percentages for the top three groups.

In [ ]:
print(sum(df1['Ethnicity'] == 'BL') / df1.shape[0] * 100)
print(sum(df1['Ethnicity'] == 'WH') / df1.shape[0] * 100)
print(sum(df1['Ethnicity'] == 'HS') / df1.shape[0] * 100)
print(sum(df1['Ethnicity'] == 'OI') / df1.shape[0] * 100)

**5.3:** The following bar graph displays the number of individuals within each ethnic group represented in the census. In order to clearly communicate what each abbreviation represents; another custom function was developed to recode the labels to their full titles.  

In [ ]:
df1["Ethnicity"].value_counts().plot(kind='bar', xlabel='Ethnicity', ylabel='Frequency')

**5.4:** A new dataframe was created for this process. The output displays the new labels for each ethnic group along with the number of individuals belonging to each of them.

In [ ]:
df_eth = pd.DataFrame(df1[["ID_Number", "Ethnicity"]])

def add_column_eth(row):
    if row["Ethnicity"] == 'BL': cat = "Black"
    elif row["Ethnicity"] == 'WH': cat = "White"
    elif row["Ethnicity"] == 'HS': cat = "Hispanic"
    elif row["Ethnicity"] == 'OA': cat = "Asian"
    elif row["Ethnicity"] == 'HW': cat = "Hawaiian/Pacific Islander"
    elif row["Ethnicity"] == 'HB': cat = "Black Hispanic"
    elif row["Ethnicity"] == 'OI': cat = "American Indian"
    return cat

df_eth["Ethnicity"] = df_eth.apply(add_column_eth, axis = 1)
df_eth["Ethnicity"].value_counts()

### Step 6: Total Arrests by County
**6.1:** Another avenue to explore was the number of incarcerated individuals differed depending on their county of admission. A third dataframe was created and used to count the number of entries corresponding to each county.

In [ ]:
df_county = pd.DataFrame(df1[["ID_Number", "County_of_Admission", "Term_at_Admission", "Total_Max_Term"]])
print(df_county['County_of_Admission'].value_counts())

**6.2:** One possible avenue is to explore is how prison term at admission differed between each county. To answer this question, the table below was created to crosstab the county of admission variable and term at admission variable. Further below is a table that converts the raw numbers into percentages of the total arrests in that county.

In [ ]:
pd.crosstab(df_county["County_of_Admission"], df_county["Term_at_Admission"], margins=True)

### Step 7: Closing the SQL Database Connection (Optional)

In [ ]:
#cnx.close()
#cursor.close()